In [1]:
import sys
import os
from pathlib import Path

# Add the project root to Python path so we can import src
# Get current working directory
cwd = Path().resolve()

# If we're in the notebooks directory, go up one level
# Otherwise, assume we're already in the project root
if cwd.name == "notebooks":
    project_root = cwd.parent
else:
    project_root = cwd

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✓ Project root added to path:", project_root)
print("✓ Current working directory:", Path.cwd())

✓ Project root added to path: /Users/rajatsharma/Desktop/MultiModalRAG/server
✓ Current working directory: /Users/rajatsharma/Desktop/MultiModalRAG/server/notebooks


In [2]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_tavily import TavilySearch
from src.rag.retrieval.index import retrieve_context
from src.rag.retrieval.utils import prepare_prompt_and_invoke_llm
from src.config.index import appConfig
import json
from langgraph.graph import MessagesState
from typing import Any, List, Dict
from typing_extensions import Annotated
from langgraph.types import Command
from langchain_core.tools.base import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableConfig
from datetime import datetime

In [3]:
# Custom Agent State to track citations
class CustomAgentState(MessagesState):
	"""Extended agent state with citations tracking"""
	citations: Annotated[List[Dict[str, Any]], lambda x, y: x + y] = []

In [4]:
# Create a RAG search tool bound to a specific project
def create_rag_tool(project_id: str):
	"""Create a RAG search tool bound to a specific project"""
	
	@tool
	def rag_search(
		query: str,
		tool_call_id: Annotated[str, InjectedToolCallId],
	) -> Command:
		"""
		Search through project documents using RAG (Retrieval-Augmented Generation).
		This tool retrieves relevant context from the current project's documents based on the query.
		
		Args:
			query: The search query or question to find relevant information
			
		Returns:
			A formatted string containing the retrieved context and answer based on the documents
		"""
		try:
			# Retrieve context using the existing RAG pipeline
			texts, images, tables, citations = retrieve_context(project_id, query)
			
			# If no context found, return a message
			if not texts and not images and not tables:
				return Command(
					update={
						"messages": [
							ToolMessage(
								"No relevant information found in the project documents for this query.",
								tool_call_id=tool_call_id
							)
						]
					}
				)
				
			# Prepare the response using the existing LLM preparation function
			response = prepare_prompt_and_invoke_llm(
				user_query=query,
				texts=texts,
				images=images,
				tables=tables
			)
			
			return Command(
				update={
					"messages": [
						ToolMessage(
							content=response,
							tool_call_id=tool_call_id
						)
					],
					"citations": citations
				}
			)       
		except Exception as e:
			return Command(
				update={
					"messages": [
						ToolMessage(
							f"Error retrieving information: {str(e)}",
							tool_call_id=tool_call_id
						)
					]
				}
			)

	return rag_search

In [5]:
# Create a RAG agent with a specific project
def create_rag_agent(project_id: str, model: str = "gpt-4o"):
	"""Create an agent with RAG tool for a specific project"""
	
	tools = [create_rag_tool(project_id)]
	
	system_prompt = """You are a helpful AI assistant with access to a RAG (Retrieval-Augmented Generation) tool that searches project-specific documents.

For every user question:

1. Do not assume any question is purely conceptual or general.  
2. Use the `rag_search` tool immediately with a clear and relevant query derived from the user’s question.  
3. Carefully review the retrieved documents and base your entire answer on the RAG results.  
4. If the retrieved information fully answers the user’s question, respond clearly and completely using that information.  
5. If the retrieved information is insufficient or incomplete, explicitly state that and provide helpful suggestions or guidance based on what you found.  
6. Always present answers in a clear, well-structured, and conversational manner.

**Never answer without first querying the RAG tool. This ensures every response is grounded in project-specific context and documentation.**"""
	
	agent = create_agent(
		model=model,
		tools=tools,
		system_prompt=system_prompt,
		state_schema=CustomAgentState
	)
	
	return agent

In [6]:
# Create a web search agent
def create_web_search_agent(model: str = "gpt-4o", use_tavily: bool = True):
	"""Create an agent with web search capabilities"""
	
	# Choose search tool based on availability
	if use_tavily and os.getenv("TAVILY_API_KEY"):
		search_tool = TavilySearch(max_results=5, search_depth="advanced")
	
	tools = [search_tool]

	current_date = datetime.now().strftime("%B %d, %Y")  # e.g., "November 25, 2025"

	system_prompt = f"""You are a specialized web search assistant.
	Your job is to search the internet for current information and provide accurate, up-to-date answers.

	**Current Date: {current_date}**

	For every query you receive:
	1. **Reformulate vague queries into specific search terms** before searching
	2. Use the web search tool with clear, specific queries
	3. Synthesize information from multiple search results when possible
	4. Provide clear, factual answers with context
	5. Indicate the recency and reliability of information when relevant

	**Query Reformulation Examples:**
	- "What's trending on social media today?" → Try: "Twitter trending topics today" OR "viral news today"
	- "Today's top headlines" → Try: "breaking news today" OR "top news stories {current_date}"
	- "What's happening in tech?" → Try: "latest tech news today" OR "technology headlines today"
	- Add date context when relevant (e.g., "news {current_date}")

	**If initial search returns insufficient or irrelevant results:**
	1. Rephrase the query with more specific terms (e.g., add location, date, or focus area)
	2. Try searching with alternative keywords or synonyms
	3. Make 2-3 search attempts with different query formulations if needed
	4. If still unsuccessful, clearly state what you found vs. what was requested

	Focus on current events, general knowledge, and information not available in internal documents.
	Never fabricate information - only use what's found in search results."""
	
	agent = create_agent(
		model=model,
		tools=tools,
		system_prompt=system_prompt,
		state_schema=CustomAgentState
	)
	
	return agent

In [7]:
from typing import Optional

# Wrap sub-agents as tools for the supervisor
def create_supervisor_tools(project_id: str, model: Optional[str] = "gpt-4o"):
	"""Create supervisor tools that wrap the specialized agents"""
	
	# Create the specialized agents
	rag_agent = create_rag_agent(project_id, model)
	web_agent = create_web_search_agent(model)
	
	@tool
	# def search_project_documents(query: str) -> str:
	def rag_search(
		query: str,
		tool_call_id: Annotated[str, InjectedToolCallId],
	) -> Command:
		"""Search internal project documents using RAG.
		
		Use this when the user asks about:
		- Project-specific information
		- Internal documentation
		- Previously uploaded files and documents
		- Company/project-specific data
		- Technical specifications from project files
		
		Args:
			query: Natural language query about project documents
			
		Returns:
			Relevant information from project documents with citations
		"""
		result = rag_agent.invoke({
			"messages": [{"role": "user", "content": query}]
		})

		final_message = result["messages"][-1]
		content = final_message.content if hasattr(final_message, 'content') else str(final_message)
		citations = result.get("citations", [])
		
		# # Extract the final response
		# final_message = result["messages"][-1]
		# if hasattr(final_message, 'content'):
		# 	return final_message.content
		# return str(final_message)
		# Return Command that updates both messages AND citations
		return Command(
			update={
				"messages": [
					ToolMessage(
						content=content,
						tool_call_id=tool_call_id
					)
				],
				"citations": citations  # ✅ Propagate citations!
			}
		)
	
	@tool
	def search_web(query: str) -> str:
		"""Search the internet for current information.
		
		Use this when the user asks about:
		- Current events or recent news
		- General knowledge not in project documents
		- External information or public data
		- Market trends or industry news
		- Any information that requires up-to-date web sources
		
		Args:
			query: Natural language query for web search
			
		Returns:
			Relevant information from web search results
		"""
		result = web_agent.invoke({
			"messages": [{"role": "user", "content": query}]
		})
		
		# Extract the final response
		final_message = result["messages"][-1]
		if hasattr(final_message, 'content'):
			return final_message.content
		return str(final_message)
	
	return [rag_search, search_web]

In [8]:
# Create a supervisor agent
def create_supervisor_agent(project_id: str, model: str = "gpt-4o"):
	"""Create a supervisor agent that coordinates RAG and web search agents"""
	
	# Get the supervisor tools (wrapped agents)
	tools = create_supervisor_tools(project_id, model)

	current_date = datetime.now().strftime("%B %d, %Y")  # "November 25, 2025"
	
	system_prompt = f"""You are an intelligent supervisor assistant that coordinates between two specialized agents:

**Current Date: {current_date}**


### Available Agents

1. **Project Documents Agent** (rag_search):
   - Searches internal project documents using RAG
   - Use for project-specific queries, internal documentation, uploaded files

2. **Web Search Agent** (search_web):
   - Searches the internet for current information
   - Use for current events, general knowledge, external information
   - ONLY use this tool if asked by the user or mentioned in the question

### Core Responsibilities

- Analyze user queries and determine which agent(s) to use
- Route queries to the appropriate agent(s) — you MUST NOT answer substantive questions directly
- For complex queries, coordinate multiple agents in sequence
- Synthesize results from multiple agents into coherent answers
- Prioritize project documents for project-specific questions
- Use web search ONLY if asked by the user or mentioned in the question

### Query Routing Rules

**ALWAYS use tools for:**
- Any question requiring factual information
- Project-specific queries
- Technical questions
- Current events or news
- General knowledge questions
- Analysis or research requests

**Direct response permitted ONLY for:**
- Simple greetings (hi, hello, how are you)
- Acknowledgments (thanks, ok, got it)
- Basic clarification requests about your capabilities
- Farewell messages (goodbye, bye)

**ALWAYS use the RAG tool for the questions**
**Return as much information that is given from the RAG tool as possible to the user**

For all other queries, you MUST route to the appropriate agent(s) and synthesize their responses. Your role is coordination and synthesis, not direct knowledge provision.
"""
	
	supervisor = create_agent(
		model=model,
		tools=tools,
		system_prompt=system_prompt,
		state_schema=CustomAgentState
	)
	
	return supervisor

In [9]:
project_id = "4fb3dfca-13d3-45ca-8cb2-5e7192c43904"
supervisor = create_supervisor_agent(project_id, model="gpt-4o")

In [11]:
content = "Net worth of Elon Musk?"
inputs = {"messages": [{"role": "user", "content": content}]}

# Stream the response
for chunk in supervisor.stream(inputs, stream_mode="updates"):
	for step, data in chunk.items():
		print(f"step: {step}")
		print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'name': 'search_web', 'args': {'query': 'Elon Musk net worth December 2025'}, 'id': 'call_GWCKW1d8X9ug8Qjr1rKNqO2x'}]
step: tools
content: [{'type': 'text', 'text': "As of December 2025, Elon Musk's net worth is estimated to be approximately $497.4 billion. This makes him the richest person on Earth during this period. His wealth primarily stems from his significant holdings in companies like Tesla and SpaceX. \n\nThis information is based on recent financial analyses and news reports."}]
step: model
content: [{'type': 'text', 'text': "As of December 2025, Elon Musk's net worth is estimated to be approximately $497.4 billion, making him the richest person on Earth at this time. His wealth primarily comes from his significant investments in companies such as Tesla and SpaceX."}]
